## Load trained GNN

In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import re
import itertools
import sys
sys.path.append("/Users/raffaelepojer/Dev/RBN-GNN/python/")
from gnn.ACR_graph import *

In [2]:
model = MYACRGnnGraph(
    input_dim=7,
    hidden_dim=[10, 8, 6],
    num_layers=3,
    mlp_layers=0,
    final_read="add",
    num_classes=2,
    fwd_dp=0.1,
    lin_dp=0.1,
    mlp_dp=0.3
)

criterion = nn.CrossEntropyLoss()
model.load_state_dict(torch.load(f"/Users/raffaelepojer/Dev/RBN-GNN/models/triangle_10_8_6_20230725-152135/exp_41/rbn_acr_graph_triangle_10_8_6_add.pt"))
model.eval()

MYACRGnnGraph(
  (layers): ModuleList(
    (0-2): 3 x MYACRConv()
  )
  (linear): Linear(in_features=6, out_features=2, bias=True)
)

In [3]:
def extract_edges(file_path):
    file = open(file_path, "r")
    lines = []
    for x in file:
        lines.append(x.strip())

    lambda1 = lambda s: int(str(s[-1]))
    features = []
    first = 0
    for i, l in enumerate(lines):
        if l == "<ProbabilisticRelsCase>":
            first = i
        if i > first and "edge" in l and "true" in l:
            for match in re.findall(r'(?<=\().*?(?=\))',l):
                a,b = map(lambda1, match.split(','))
                features.append([a, b])
                features.append([b, a])
            break
    features.sort()

    if len(features) > 0:
        return torch.tensor(list(k for k,_ in itertools.groupby(features))).t()
    else:
        a = torch.tensor([[]], dtype=torch.long)
        b = torch.tensor([[]], dtype=torch.long)
        edge_index = torch.cat((a, b), 0)
        return edge_index

def count_nodes(file_path):
    file = open(file_path, "r")
    lines = []
    for x in file:
        lines.append(x.strip())
    lambda1 = lambda s: int(s[-1])
    nodes = []
    first = 0
    for i, l in enumerate(lines):
        if l == "<PredefinedRels>":
            first = i
        if i > first and "node" in l and "true" in l:
            regex = r"(?<=\()[a-zA-Z0-9_]+(?=\))"
            match = re.findall(regex, "".join(l))
            nodes = list(map(lambda1, match))
    return len(nodes)

def extract_features(file_path):
    features_name = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    file = open(file_path, "r")
    lines = []
    for x in file:
        lines.append(x.strip())

    lambda1 = lambda s: int(s[-1])
    all_features = []
    num_nodes = count_nodes(file_path)
    for i in range(num_nodes):
        all_features.append([0.0]*7)

    first = 0
    for i, l in enumerate(lines):
        if l == "<ProbabilisticRelsCase>":
            first = i
        
        match = re.search(r'rel="([^"]*)"', l)
        for j, feature in enumerate(features_name):
            if i > first and match and match.group(1) in features_name and match.group(1) in feature and "true" in l:
                regex = r"(?<=\()[a-zA-Z0-9_]+(?=\))"
                match = re.findall(regex, "".join(l))
                nodes = list(map(lambda1, match))
                for idx in nodes:
                    all_features[idx][j] = 1.0
                break
            
    return(all_features)

# Test probabilities

In [28]:
file_path = f"/Users/raffaelepojer/Dev/RBN-GNN/models/triangle_10_8_6_20230725-152135/exp_41/graphs/10_restarts/0_graphs/edge_030/n8.rdef"

edge_index = torch.tensor(extract_edges(file_path))
print(edge_index)

x = torch.tensor(extract_features(file_path))
batch = torch.zeros(x.size()[0]).type(torch.LongTensor)

out = model(x, edge_index, batch)
m = nn.Softmax(dim=1)
print(m(out))

tensor([[0, 1, 2, 3, 3, 4, 5, 5],
        [5, 3, 5, 1, 4, 3, 0, 2]])
tensor([[0.8870, 0.1130]], grad_fn=<SoftmaxBackward0>)


/var/folders/cx/q4nyk81514qb1xtxy82fkxhm0000gn/T/ipykernel_93056/156781511.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(extract_edges(file_path))


In [28]:
probs = []
for i in range(2, 9):
    file_path = f"/Users/raffaelepojer/Dev/RBN-GNN/models/triangle_10_8_6_20230725-152135/exp_41/graphs/10_restarts/0_graphs/edge_030/n"+str(i)+".rdef"

    edge_index = torch.tensor(extract_edges(file_path))

    x = torch.tensor(extract_features(file_path))
    batch = torch.zeros(x.size()[0]).type(torch.LongTensor)

    out = model(x, edge_index, batch)
    m = nn.Softmax(dim=1)
    probs.append(m(out)[0][0].item())

print(probs)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define the number of nodes for each bar
num_nodes = list(range(2, 10))

# Set LaTeX style for the plot
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# Create the bar chart
plt.figure(figsize=(8, 6), dpi=100)  # Adjust the figure size and dpi as needed
sns.barplot(x=num_nodes, y=probs, color='skyblue')

# Add labels and title
plt.xlabel('Number of Nodes')
plt.ylabel('Probability')
plt.title('Probability of Each Graph with Respect to Number of Nodes')

# Increase the number of ticks on the y-axis (probability)
plt.yticks(np.arange(0, 1.1, 0.1))

# Add a light grid for better readability
plt.grid(axis='y', color='gray', linestyle=':', linewidth=0.5)

# Add probabilities as text above each bar
for i, prob in enumerate(probs):
    plt.text(i, prob + 0.01, f'{prob:.4f}', ha='center')

# Save the plot as a high-resolution image (e.g., PNG format)
plt.savefig('probability_bar_chart.png', dpi=300, bbox_inches='tight')

# Show the plot (optional)
plt.show()

[0.026286710053682327, 0.012315792962908745, 0.019411813467741013, 0.0437067411839962, 0.8869744539260864, 0.9391584396362305, 0.9592694640159607]


/var/folders/cx/q4nyk81514qb1xtxy82fkxhm0000gn/T/ipykernel_77552/1801593941.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(extract_edges(file_path))


ValueError: Grouper for '2' not 1-dimensional

<Figure size 800x600 with 0 Axes>